In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
from os import path
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from models import *
from etl import *
import plotly.plotly as py
import plotly.graph_objs as go

Database session created


In [21]:
df = pd.DataFrame([{'content_url':content.content_url, 'image_url':content.image_url,'title':content.title
                             , 'published': content.published, 'length': content.length, 'medium':content.medium.name,
                             'provider': content.provider.name, 'api_id':content.provider.api_id, 'formality':content.provider.formality.type,
                             'category':content.category.name} for content in session.query(Content).all()])
df.to_csv('all_content.csv')

In [22]:
df

,api_id,category,content_url,formality,image_url,length,medium,provider,published,title
0,engadget,keto,https://www.engadget.com/2018/11/27/keyto-brea...,Intermediate,https://o.aolcdn.com/images/dims?thumbnail=120...,2.00,text,Engadget,2018-11-27,Keyto breath analyzer promises to keep your ke...
1,usa-today,keto,https://www.usatoday.com/story/life/grateful/2...,Intermediate,https://www.gannett-cdn.com/presto/2018/10/17/...,5.00,text,USA Today,2018-11-16,An irreverent Thanksgiving: Making the holiday...
2,the-wall-street-journal,keto,https://www.wsj.com/articles/this-coffee-is-ni...,Intermediate,https://si.wsj.net/public/resources/images/B3-...,6.00,text,The Wall Street Journal,2018-11-26,This Coffee Is Nice But It Needs More Butter -...
3,the-telegraph,keto,https://www.telegraph.co.uk/boxing/2018/11/27/...,Intermediate,https://www.telegraph.co.uk/content/dam/boxing...,1.00,text,The Telegraph,2018-11-27,Revealed: The secrets behind Tyson Fury's extr...
4,newsweek,keto,https://www.newsweek.com/vinny-guadagnino-talk...,Intermediate,https://s.newsweek.com/sites/www.newsweek.com/...,3.00,text,Newsweek,2018-11-15,What Happens When 'Jersey Shore' Ends? Vinny G...
5,buzzfeed,keto,https://www.buzzfeed.com/michelleno/low-carb-k...,Intermediate,https://img.buzzfeed.com/buzzfeed-static/stati...,4.00,text,Buzzfeed,2018-11-17,14 Keto Versions Of Popular Holiday Desserts
6,business-insider,keto,https://www.businessinsider.com/best-keto-meal...,Intermediate,https://amp.businessinsider.com/images/5bfc676...,1.00,text,Business Insider,2018-11-27,10 of the best keto meal options at popular ch...
7,business-insider,keto,https://www.businessinsider.com/keto-breathaly...,Intermediate,https://amp.businessinsider.com/images/5bfd48a...,3.00,text,Business Insider,2018-11-27,A new $100 keto breathalyzer could finally mak...
8,business-insider,keto,https://www.businessinsider.com/keto-thanksgiv...,Intermediate,https://amp.businessinsider.com/images/5bf2f66...,1.00,text,Business Insider,2018-11-20,"How to have a carb-free keto Thanksgiving, acc..."
9,buzzfeed,keto,https://www.buzzfeed.com/jamedjackson/20-hilar...,Intermediate,https://img.buzzfeed.com/buzzfeed-static/stati...,4.00,text,Buzzfeed,2018-11-27,"These ""Overheard"" Tweets Will Make You Wonder ..."


In [38]:
df[['title','formality','medium']].groupby(['formality','medium']).count()

title
formality    medium       
Formal       audio    1978
             text     3067
             video     117
Informal     text     1501
             video       1
Intermediate audio    6963
             text     1641
             video     784

In [2]:
plot_stacked_OVERALL()

# Ketogenic

In [3]:
Limit_widget1 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget1 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)
expertise_widget1 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget1 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget1, Medium = Medium_widget1, search_param=topic_widget1, Formality = expertise_widget1, Max_Length = max_length_widget1);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [4]:
alt_widget_1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_1);

interactive(children=(Dropdown(description='Topic', options=('keto', 'mct', 'natural', 'corn-fed', 'cruciferou…

# Paleolithic

In [5]:
Limit_widget2 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget2 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)
expertise_widget2 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget2 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget2, Medium = Medium_widget2, search_param=topic_widget2, Formality = expertise_widget2, Max_Length = max_length_widget2);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [6]:
alt_widget_2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_2);

interactive(children=(Dropdown(description='Topic', index=15, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegan

In [7]:
Limit_widget3 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget3 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)
expertise_widget3 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget3 = widgets.IntSlider(min=0, max=100, step=10, value=20)


interact(query_content,Limit = Limit_widget3, Medium = Medium_widget3, search_param=topic_widget3, Formality = expertise_widget3, Max_Length = max_length_widget3);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [8]:
alt_widget_3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_3);

interactive(children=(Dropdown(description='Topic', index=13, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegetarian

In [9]:
Limit_widget4 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget4 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)
expertise_widget4 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget4 = widgets.IntSlider(min=0, max=200, step=10, value=20)


interact(query_content,Limit = Limit_widget4, Medium = Medium_widget4, search_param=topic_widget4, Formality = expertise_widget4, Max_Length = max_length_widget4);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [10]:
alt_widget_4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_4);

interactive(children=(Dropdown(description='Topic', index=14, options=('keto', 'mct', 'natural', 'corn-fed', '…

# General

In [11]:
Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)
expertise_widget5 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget5 = widgets.IntSlider(min=0, max=300, step=10, value=20)


interact(query_content,Limit = Limit_widget5, Medium = Medium_widget5, search_param=topic_widget5, Formality = expertise_widget5, Max_Length = max_length_widget5);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [12]:
alt_widget_5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_5);

interactive(children=(Dropdown(description='Topic', options=('general',), value='general'), Output()), _dom_cl…

# Search

In [13]:
# search_widget = widgets.Text(placeholder = 'Search News', description='Search:', disabled=False,continuous_update=False)
# Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
# Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')




# interact(query_content,search_param=search_widget,Limit = Limit_widget5, Medium = Medium_widget5);